## Kütüphanelerin kurulumu

In [ ]:
!pip install -q flwr[simulation] torch torchvision matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.4/200.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 44.6 MB/s eta 0:00:00


In [ ]:
!pip install torchattacks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstall

## Kütüphanelerin çağırılması

In [ ]:
from collections import OrderedDict
from typing import List, Tuple
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST, CIFAR10
from torchsummary import summary
import flwr as fl
from flwr.common import Metrics
import torchattacks
from torchattacks import PGD

## Veri setinin yüklenmesi, normalize edilmesi, clientlere ayırılması

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:03<00:00, 45367399.69it/s]


Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


In [ ]:
NUM_CLIENTS = 50

In [ ]:
BATCH_SIZE = 8

def load_datasets(trainset, testset):

    train_partition_size = len(trainset) // NUM_CLIENTS
    train_lengths = [train_partition_size] * NUM_CLIENTS
    train_datasets = random_split(trainset, train_lengths, torch.Generator().manual_seed(42))
    test_partition_size = len(testset) // NUM_CLIENTS
    test_lengths = [test_partition_size] * NUM_CLIENTS
    test_datasets = random_split(testset, test_lengths, torch.Generator().manual_seed(42))

    testloaders = []
    trainloaders = []
    for train_ds, test_ds in zip(train_datasets, test_datasets):
        trainloaders.append(DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True))
        testloaders.append(DataLoader(test_ds, batch_size=BATCH_SIZE))
    return trainloaders, testloaders

trainloaders, testloaders = load_datasets(trainset, testset)

## Mimarinin tanımlanması

In [ ]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
summary(Net(), (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 62,006
Trainable params: 62,006
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.31
----------------------------------------------------------------


## train, test ve eval_robust fonksiyonlarının tanımlanması

Atak eval_robust fonksiyonu içerisindeki PGD fonksiyonu ile yapılmıştır.

In [ ]:
def train(net, trainloader, epochs: int, verbose=False):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

def eval_robust(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.enable_grad():
        for images, labels in testloader:
            atk = PGD(net, eps=8/255, alpha=2/225, steps=10, random_start=True)
            adv_images = atk(images, labels)
            outputs = net(adv_images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

## Sunucudan parametreleri almak ve sunucuya parametreleri göndermek için fonksiyonların tanımlanması

In [ ]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

## Client sınıfının oluşturulması

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, testloader):
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=5)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss1, robust = eval_robust(self.net, self.testloader)
        loss2, clean = test(self.net, self.testloader)
        return float(loss2), len(self.testloader), {"clean_acc": float(clean), "robust_acc": float(robust), "robust_loss": float(loss1)}

## Clientleri çağırma fonksiyonun tanımlanması

In [ ]:
def client_fn(cid: str) -> FlowerClient:

    net = Net().to()

    trainloader = trainloaders[int(cid)]
    testloader = testloaders[int(cid)]

    return FlowerClient(net, trainloader, testloader)

## Loss dışındaki metriklerin hesaplanması için tanımlanan fonksiyon

In [ ]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    cleans = [num_examples * m["clean_acc"] for num_examples, m in metrics]
    robusts = [num_examples * m["robust_acc"] for num_examples, m in metrics]
    losses = [num_examples * m["robust_loss"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    return {"clean_acc": sum(cleans) / sum(examples), "robust_acc": sum(robusts) / sum(examples), "robust_loss": sum(losses) / sum(examples)}

## Aggregation algoritmasının belirlenmesi ve simülasyonun yapılması

In [ ]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.2,
    fraction_evaluate=0.1,
    min_fit_clients=10,
    min_evaluate_clients=5,
    min_available_clients=int(NUM_CLIENTS*0.50),
    evaluate_metrics_aggregation_fn= weighted_average,
)

client_resources = {"num_cpus": 1, "num_gpus": 0.0}


fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-10-28 09:34:50,146 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-10-28 09:34:54,460	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-10-28 09:34:58,123 | app.py:210 | Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7829839872.0, 'object_store_memory': 3914919936.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7829839872.0, 'object_store_memory': 3914919936.0, 'CPU': 2.0}
INFO flwr 2023-10-28 09:34:58,129 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO:flwr:Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO flwr 2023-10-28 09:34:58,207 | app.py:270 | Flower VCE: Creating 

History (loss, distributed):
	round 1: 0.2731341938972473
	round 2: 0.2188479949235916
	round 3: 0.20187671494483947
	round 4: 0.1822571063041687
	round 5: 0.18732131618261338
	round 6: 0.17916770863533021
	round 7: 0.17768100202083587
	round 8: 0.17611660353839398
	round 9: 0.17311891168355942
	round 10: 0.17431776762008666
History (metrics, distributed, evaluate):
{'clean_acc': [(1, 0.202), (2, 0.375), (3, 0.417), (4, 0.462), (5, 0.462), (6, 0.493), (7, 0.485), (8, 0.509), (9, 0.511), (10, 0.533)], 'robust_acc': [(1, 0.13), (2, 0.106), (3, 0.081), (4, 0.07), (5, 0.079), (6, 0.069), (7, 0.057), (8, 0.053), (9, 0.024), (10, 0.034)], 'robust_loss': [(1, 0.2850013771057129), (2, 0.30676017606258393), (3, 0.34053148281574247), (4, 0.3833031451702118), (5, 0.43238155925273897), (6, 0.46894561672210694), (7, 0.5243183467388153), (8, 0.5611490321159363), (9, 0.6310246341228485), (10, 0.6891230792999268)]}